In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import os

In [5]:
# num2label 파일 로드
num2label = "./utils/dict_num_to_label.pkl"
with open(num2label, "rb") as f:
    n2l = pickle.load(f)

In [6]:
import re

model_paths = os.listdir("/data/ephemeral/level2-klue-nlp-06/outputs")

In [7]:
model_paths

['output_75.8300.csv',
 'google_rembert_type_punct_prompt_submission_68.8709.csv',
 'output_76.2207.csv',
 'ensemble_76.4762.csv',
 'output_72.1076.csv',
 'output_75.6213.csv',
 'output_72.1668.csv',
 'unknown_xlm-roberta-large_v03_69.86.csv',
 'output_75.0000.csv',
 'hfl_cino-large-v2_typed_punct_marker_submission_68.1797.csv',
 'output_76.2142.csv',
 'output_76.3133.csv',
 'output_72.8648.csv',
 'output_73.5101.csv',
 'output_73.1501.csv',
 'klue_roberta-large_typed_punct_marker-5_concat-prompt_focal_submission_72.8704.csv',
 'studio-ousia_mluke-large-lite_typed_punct_marker_submission_69.321.csv',
 'sota_output_76.3133.csv',
 'wooy0ng_korquad1-klue-roberta-large_typed_punct_marker_submission_71.7003.csv']

### 자동으로 랜덤 8개 조합해서 점수 반환, 가장 높은 점수의 조합을 최종적으로 반환

In [14]:
# 파일 경로에서 숫자 부분 추출
weights = [float(re.search(r'\d+\.\d+', path).group()) for path in model_paths]
# weights = [50., 80., 100., 80., 90., 50.]

# submission 파일 로드
dfs = [pd.read_csv("/data/ephemeral/level2-klue-nlp-06/outputs/" + path) for path in model_paths]

# probs 가중 평균
probs = []
for row in zip(*[df["probs"].tolist() for df in dfs]):
    temp = []
    for col in zip(*[eval(p) for p in row]):
        temp.append(sum(float(r*w/100) for r,w in zip(col,weights)))
    probs.append(temp)

pred_label = [n2l[i.index(max(i))] for i in probs]

df = pd.DataFrame(columns=["id", "pred_label", "probs"])
df["id"] = range(0, len(pred_label))
df["pred_label"] = pred_label
df["probs"] = probs



In [15]:
import random
import tqdm

In [16]:
model_paths = random.sample(os.listdir("./outputs"), 8)
model_paths

['hfl_cino-large-v2_typed_punct_marker_submission_68.1797.csv',
 'output_75.6213.csv',
 'wooy0ng_korquad1-klue-roberta-large_typed_punct_marker_submission_71.7003.csv',
 'unknown_xlm-roberta-large_v03_69.86.csv',
 'ensemble_76.4762.csv',
 'output_76.3133.csv',
 'studio-ousia_mluke-large-lite_typed_punct_marker_submission_69.321.csv',
 'sota_output_76.3133.csv']

In [17]:
[float(re.search(r'\d+\.\d+', path).group()) for path in model_paths]

[68.1797, 75.6213, 71.7003, 69.86, 76.4762, 76.3133, 69.321, 76.3133]

In [18]:
str([50.0, 80.0, 100.0, 80.0, 90.0, 50.0, 80.0, 90.0])

'[50.0, 80.0, 100.0, 80.0, 90.0, 50.0, 80.0, 90.0]'

In [19]:
from tqdm import tqdm

In [20]:
# for문을 돌면서 model_paths에서 자동으로 랜덤 8개 앙상블해서 점수 반환, 가장 높은 점수의 조합을 최종적으로 반환

loop = 100
best_score = 0

for i in tqdm(range(loop)):

    # 파일 경로에서 8개 랜덤 추출
    model_paths = random.sample(os.listdir("./outputs"), 8)


    # 파일 경로에서 숫자 부분 추출
    weights = [float(re.search(r'\d+\.\d+', path).group()) for path in model_paths]
    # weights = [50., 80., 100., 80., 90., 50.]

    # submission 파일 로드
    dfs = [pd.read_csv("/data/ephemeral/level2-klue-nlp-06/outputs/" + path) for path in model_paths]

    # probs 가중 평균
    probs = []
    for row in zip(*[df["probs"].tolist() for df in dfs]):
        temp = []
        for col in zip(*[eval(p) for p in row]):
            temp.append(sum(float(r*w/100) for r,w in zip(col,weights)))
        probs.append(temp)

    pred_label = [n2l[i.index(max(i))] for i in probs]

    df = pd.DataFrame(columns=["id", "pred_label", "probs"])
    df["id"] = range(0, len(pred_label))
    df["pred_label"] = pred_label
    df["probs"] = probs

    f1, _ = gift_box(df)

    if f1 > best_score:
        best_score = f1
        best_model_paths = model_paths
        print(f"best_score : {best_score}, best_model_paths : {best_model_paths}")

  1%|          | 1/100 [00:06<10:44,  6.51s/it]

best_score : 75.67394094993583, best_model_paths : ['output_76.3133.csv', 'output_75.0000.csv', 'sota_output_76.3133.csv', 'studio-ousia_mluke-large-lite_typed_punct_marker_submission_69.321.csv', 'output_72.8648.csv', 'wooy0ng_korquad1-klue-roberta-large_typed_punct_marker_submission_71.7003.csv', 'output_73.1501.csv', 'google_rembert_type_punct_prompt_submission_68.8709.csv']


  2%|▏         | 2/100 [00:13<10:39,  6.52s/it]

best_score : 75.82452769772655, best_model_paths : ['ensemble_76.4762.csv', 'output_72.8648.csv', 'sota_output_76.3133.csv', 'hfl_cino-large-v2_typed_punct_marker_submission_68.1797.csv', 'output_76.3133.csv', 'output_72.1668.csv', 'output_73.1501.csv', 'output_75.6213.csv']


  9%|▉         | 9/100 [00:59<09:59,  6.58s/it]

...

 49%|████▉     | 49/100 [05:21<05:33,  6.54s/it]

best_score : 76.02057216329153, best_model_paths : ['output_73.5101.csv', 'sota_output_76.3133.csv', 'unknown_xlm-roberta-large_v03_69.86.csv', 'output_76.2207.csv', 'output_75.0000.csv', 'output_72.1076.csv', 'output_72.1668.csv', 'output_72.8648.csv']


 74%|███████▍  | 74/100 [08:05<02:50,  6.55s/it]

best_score : 76.07303010890453, best_model_paths : ['output_76.3133.csv', 'sota_output_76.3133.csv', 'output_72.8648.csv', 'output_72.1668.csv', 'output_76.2207.csv', 'unknown_xlm-roberta-large_v03_69.86.csv', 'output_72.1076.csv', 'wooy0ng_korquad1-klue-roberta-large_typed_punct_marker_submission_71.7003.csv']


100%|██████████| 100/100 [10:55<00:00,  6.55s/it]
